In [1]:
import psycopg2
import os

In [4]:
os.environ['USERNAME'] = 'postgres'
os.environ['PASSWORD'] = '<password>'

In [5]:
USER = os.environ['USERNAME']
PASSWORD = os.environ['PASSWORD']

In [6]:
USER

'postgres'

In [7]:
len(PASSWORD)

12

In [8]:
conn = psycopg2.connect(
    host='192.168.4.25',database='ecomm',
                        user=USER,
                        password=PASSWORD,
                        port=5432)

In [10]:
cursor = conn.cursor()

In [11]:
cursor.execute("""
DROP TABLE IF EXISTS customer_accounts;

CREATE TABLE customer_accounts (
    account_id SERIAL PRIMARY KEY,
    account_name VARCHAR(100) NOT NULL,
    account_email VARCHAR(100) NOT NULL UNIQUE,
    region VARCHAR(50)
);
""")

In [12]:
# add data
cursor.executemany("""
INSERT INTO customer_accounts (account_name, account_email, region)
VALUES (%s, %s, %s);
""", [
    ('Alice', 'alice_account@example.com', 'USA'),
    ('Bob', 'bob_account@example.com', 'Canada'),
    ('Charlie', 'charlie_account@example.com', 'UK'),
    ('Diana', 'diana_account@example.com', 'Australia'),
    ('Eve', 'eve_account@example.com', 'India')
])

In [13]:
conn.commit()

In [14]:
# create has index on account_email column
cursor.execute("""
CREATE INDEX account_email_hash_idx ON customer_accounts USING hash(account_email);
""")

In [17]:
# Querying using the hash index
query = """
Set enable_seqscan = off;
EXPLAIN ANALYZE SELECT * FROM customer_accounts WHERE account_email = 'alice_account@example.com';
"""
cursor.execute(query)

In [18]:
# fetching and displaying the query plan
query_plan = cursor.fetchall()
for line in query_plan:
    print(line)

('Index Scan using account_email_hash_idx on customer_accounts  (cost=0.00..8.02 rows=1 width=558) (actual time=4.235..4.237 rows=1 loops=1)',)
("  Index Cond: ((account_email)::text = 'alice_account@example.com'::text)",)
('Planning Time: 0.073 ms',)
('Execution Time: 4.250 ms',)


In [19]:
query_enable_seqscan = """
SET enable_seqscan = ON;
"""
cursor.execute(query_enable_seqscan)

In [ ]:
cursor.close()
conn.close()